# How to use PyTorch LSTMs for time series regression

# Data

In [3]:
import pandas as pd

df = pd.read_csv("/Users/atichetsurakul/Desktop/JAN23/BiA/Joules_of_Siam_Data - Electricity_Consumption_Monthly.csv", index_col="Date")
df.head()

,Residental <150,Residental > 150,Residental Total,Small General,Medium General,Large General,Specific Business,Gov & Nonprofit,Agriculture Pumping,Temporary,Stand By Rate,Interruptible Rate,Free of Charge,Total,Direct Customer,Grand total
Date,,,,,,,,,,,,,,,,
2002-1,488,1036,1524,693,1415,2958,236,246,19,29,1,NaN,78,7199,128,7327
2002-2,510,1088,1597,719,1406,2797,238,260,26,31,1,76.0,74,7224,135,7359
2002-3,597,1246,1842,816,1664,3199,277,301,24,31,0,85.0,76,8316,155,8471
2002-4,662,1387,2049,863,1574,3014,290,287,23,34,0,86.0,86,8307,154,8461
2002-5,652,1338,1990,884,1670,3230,278,306,15,34,1,98.0,77,8582,149,8730


In [4]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

# fig = px.line(df, labels=dict(
#     created_at="Date", value="PM2.5 (ug/m3)", variable="Sensor"
# ))
# fig.update_layout(
#   template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
# )
# fig.show()
# fig.write_image("pm25_data.png", width=1200, height=600)

In [ ]:
# fig.update_yaxes(range = [0, 60])
# fig.show()
# fig.write_image("pm25_data_zoomed.png", width=1200, height=600)

## Create the target variable

In [5]:
target_sensor = "Grand total" # Comsumption and FT
features = list(df.columns.difference([target_sensor]))

forecast_lead = 15
target = f"{target_sensor}_lead{forecast_lead}"

df[target] = df[target_sensor].shift(-forecast_lead)
df = df.iloc[:-forecast_lead]

## Create a hold-out test set and preprocess the data

In [8]:
test_start = '2020-1'   #Date

df_train = df.loc[:test_start].copy()
df_test = df.loc[test_start:].copy()

print("Test set fraction:", len(df_test) / len(df))

Test set fraction: 0.08860759493670886


## Standardize the features and target, based on the training set

In [9]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

## Create datasets that PyTorch `DataLoader` can work with

In [10]:
import torch
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[self.target].values).float()
        self.X = torch.tensor(dataframe[self.features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [11]:
i = 27
sequence_length = 4

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)

X, y = train_dataset[i]
print(X)

tensor([[ 0.1598, -0.2793, -1.2205,  0.3228,  0.4190, -1.5506, -1.4474, -1.5106,
         -1.3918, -1.4566, -1.5513, -1.3258, -0.5652, -1.3331, -1.5636],
        [-0.0750,  0.2994, -1.2361,  0.3574, -0.2719, -1.3660, -1.5184, -1.7798,
         -1.4166, -1.5104, -1.5629, -1.4718, -0.8122, -1.2417, -1.5263],
        [ 0.5512,  0.5099, -1.2049,  0.5595,  0.0505, -1.0393, -0.9277, -0.4103,
         -1.0347, -1.0010, -1.1581, -1.0726, -0.7298, -1.1504, -1.0637],
        [ 0.2381, -0.5950, -1.2205,  0.5884, -0.3640, -1.3403, -1.2259,  0.4559,
         -0.7621, -0.6502, -1.0049, -0.8778, -0.2359, -1.1504, -1.0961]])


In [12]:
X, y = train_dataset[i + 1]
print(X)

tensor([[-0.0750,  0.2994, -1.2361,  0.3574, -0.2719, -1.3660, -1.5184, -1.7798,
         -1.4166, -1.5104, -1.5629, -1.4718, -0.8122, -1.2417, -1.5263],
        [ 0.5512,  0.5099, -1.2049,  0.5595,  0.0505, -1.0393, -0.9277, -0.4103,
         -1.0347, -1.0010, -1.1581, -1.0726, -0.7298, -1.1504, -1.0637],
        [ 0.2381, -0.5950, -1.2205,  0.5884, -0.3640, -1.3403, -1.2259,  0.4559,
         -0.7621, -0.6502, -1.0049, -0.8778, -0.2359, -1.1504, -1.0961],
        [-0.7795,  0.2994, -1.2205,  0.6230, -0.3640, -1.0284, -0.7176,  0.3154,
         -0.8277, -0.7268, -0.9875, -0.9947, -0.6475, -1.0895, -0.9262]])


In [13]:
print(df_train[features].iloc[(i - sequence_length + 1): (i + 1)])

        Agriculture Pumping  Direct Customer  Free of Charge  Gov & Nonprofit  \
Date                                                                            
2004-1             0.159799        -0.279315       -1.220532         0.322803   
2004-2            -0.075030         0.299439       -1.236125         0.357443   
2004-3             0.551182         0.509894       -1.204940         0.559510   
2004-4             0.238076        -0.594998       -1.220532         0.588377   

        Interruptible Rate  Large General  Medium General  Residental <150  \
Date                                                                         
2004-1            0.418979      -1.550586       -1.447393        -1.510605   
2004-2           -0.271857      -1.365996       -1.518386        -1.779822   
2004-3            0.050533      -1.039262       -0.927729        -0.410326   
2004-4           -0.363968      -1.340331       -1.225897         0.455852   

        Residental > 150  Residental Total  

In [14]:
from torch.utils.data import DataLoader
torch.manual_seed(99)

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

X, y = next(iter(train_loader))
print(X.shape)
print(X)

torch.Size([3, 4, 15])
tensor([[[ 3.1635e-01,  2.1935e+00, -3.5496e-02,  1.4659e+00, -8.7634e-02,
           2.2227e-01,  8.5060e-04,  4.2074e-01,  4.5492e-02,  9.1768e-02,
           5.0445e-02, -1.1085e-02,  9.3317e-02, -5.1109e-01,  2.2005e-01],
         [-1.0926e+00, -1.7409e-01, -5.1089e-02,  1.2870e+00, -5.4819e-01,
           1.6995e-01, -1.4965e-01,  4.6173e-02, -1.1777e-01, -1.0312e-01,
          -1.1147e-01, -1.4742e-01, -2.3595e-01, -6.0242e-01,  6.4092e-02],
         [-1.3275e+00,  1.4160e-01, -2.3820e-01,  1.4197e+00, -1.2851e+00,
           1.1566e-01, -1.3545e-01, -1.4111e-01, -1.8192e-01, -1.8377e-01,
          -1.5195e-01, -2.4481e-01, -4.0058e-01, -5.7198e-01,  6.7619e-03],
         [-1.3275e+00, -2.7931e-01,  7.3652e-02,  1.1426e+00, -6.8636e-01,
           2.6175e-01, -2.9164e-01,  2.1004e-01, -2.1399e-01, -1.7301e-01,
          -1.5195e-01, -2.1559e-01, -1.5363e-01, -4.8064e-01,  4.5249e-02]],

        [[ 5.5118e-01, -1.3842e+00,  1.7421e+00, -1.1552e+00, -7.3241e-

## Create the datasets and data loaders for real

Using just 4 time periods to forecast 15 time periods ahead seems challenging, so let's
use sequences of length 30 (60 minutes) instead.

The PyTorch `DataLoader` is a very convenient way to iterate through these datasets. For
the training set we'll shuffle (the rows *within* each training sequence are not
shuffled, only the order in which we draw those blocks). For the test set, shuffling
isn't necessary.

In [15]:
torch.manual_seed(101)

batch_size = 4
sequence_length = 30

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([4, 30, 15])
Target shape: torch.Size([4])


# The model and learning algorithm

In [23]:
from torch import nn

class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out


In [24]:
learning_rate = 5e-5
num_hidden_units = 3

model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train

In [25]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")

In [26]:
print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(2):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 3.985864202181498

Epoch 0
---------
Train loss: nan
Test loss: nan

Epoch 1
---------
Train loss: nan
Test loss: nan



# Evaluation

In [27]:
def predict(data_loader, model):
    """Just like `test_loop` function but keep track of the outputs instead of the loss
    function.
    """
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output

In [28]:
train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_train[ystar_col] = predict(train_eval_loader, model).numpy()
df_test[ystar_col] = predict(test_loader, model).numpy()

df_out = pd.concat((df_train, df_test))[[target, ystar_col]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

print(df_out)

        Grand total_lead15  Model forecast
Date                                      
2002-1              8860.0             NaN
2002-2              9437.0             NaN
2002-3              9146.0             NaN
2002-4              9089.0             NaN
2002-5              9344.0             NaN
...                    ...             ...
2021-5             17276.0             NaN
2021-6             16508.0             NaN
2021-7             16032.0             NaN
2021-8             16061.0             NaN
2021-9             15191.0             NaN

[238 rows x 2 columns]


In [ ]:
# fig = px.line(df_out, labels={'value': "PM2.5 (ug/m3)", 'created_at': 'Date'})
# fig.add_vline(x=test_start, line_width=4, line_dash="dash")
# fig.add_annotation(xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False)
# fig.update_layout(
#   template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
# )
# fig.show()
# fig.write_image("pm25_forecast.png", width=1200, height=600)